<a href="https://colab.research.google.com/github/Quanturistae/PQC_Project_Course/blob/main/Baby_Kyber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy.polynomial import Polynomial
from sympy import symbols, Poly
import numpy
from sympy import rem, ZZ
import math

In [ ]:
def Kyber_KeyGen(q, f, k, x):
  #input secret vector s (private key)
  temp_s = []

  for i in range(k):
      coeffs = list(map(int, input(f"Enter coefficients in decreasing order of degree separated by space - s_{i}: ").split()))
      temp = Poly(coeffs, x)
      temp_s.append(temp)

  print("Private Key as list:", temp_s)
  s = numpy.asmatrix(numpy.asanyarray(temp_s)).transpose()
  print("Private Key:", s)

  #input polynomial matrix A
  temp_A = []

  for i in range(k):
    temp_row_A = []
    for j in range(k):
      coeffs = list(map(int, input(f"Enter coefficients in decreasing order of degree separated by space - Ar_{i}{j}: ").split()))
      temp_element_A = Poly(coeffs, x)
      temp_row_A.append(temp_element_A)
    temp_A.append(temp_row_A)

  print("Matrix A as list:", temp_A)
  A = numpy.asmatrix(numpy.asanyarray(temp_A))
  print("Matrix A:", A)

  #input error vector e
  temp_e = []

  for i in range(k):
      coeffs = list(map(int, input(f"Enter coefficients in decreasing order of degree separated by space - e_{i}: ").split()))
      temp = Poly(coeffs, x)
      temp_e.append(temp)

  print("Error as list:", temp_e)
  e = numpy.asmatrix(numpy.asanyarray(temp_e)).transpose()
  print("error:", e)

  #t = As+e
  t = numpy.add(numpy.matmul(A, s), e)
  print("vector t before mod q:", t)

  t = numpy.array(t).flatten().tolist()

  for i in range(k):
    t[i] = rem(t[i], f)
    t[i] = Poly([coeff % q for coeff in t[i].as_poly().all_coeffs()], x)

  t = numpy.asmatrix(numpy.asanyarray(t)).transpose()
  print("vector t:", t)


  #return
  return(s, A, t)

In [ ]:
def Kyber_Encaps(q, f, k, x, A, t):
  #input randomizer polynomial vector r
  temp_r = []

  for i in range(k):
      coeffs = list(map(int, input(f"Enter coefficients in decreasing order of degree separated by space - r_{i}: ").split()))
      temp = Poly(coeffs, x)
      temp_r.append(temp)

  print("Randomizer polynomial vector r as list:", temp_r)
  r = numpy.asmatrix(numpy.asanyarray(temp_r)).transpose()
  print("Randomizer polynomial vector r:", r)

  #input error vector e_1
  temp_e_1 = []

  for i in range(k):
      coeffs = list(map(int, input(f"Enter coefficients in decreasing order of degree separated by space - e_1_{i}: ").split()))
      temp = Poly(coeffs, x)
      temp_e_1.append(temp)

  print("Error vector e_1 as list:", temp_e_1)
  e_1 = numpy.asmatrix(numpy.asanyarray(temp_e_1)).transpose()
  print("Error vector e_1:", e_1)

  #input error polynomial e_2
  e_2 = Poly(list(map(int, input(f"Enter coefficients in decreasing order of degree separated by space - e_2_{i}: ").split())), x)

  #input message in decimal to encrypt
  message_decimal = int(input("Enter message in decimal to encrypt: "))

  message_binary = bin(message_decimal)
  message_binary_list = list(message_binary[2:])
  message_binary_list_int = [int(bit) for bit in message_binary_list]
  print("Message in binary:", message_binary_list_int)
  message_binary_list_int_halfq = [(message_binary_list_int[i] * math.ceil(q/2)) for i in range(len(message_binary_list_int))]
  print("Message in binary multiplied by ceil(q/2):", message_binary_list_int_halfq)
  message_poly = Poly(message_binary_list_int_halfq, x)
  print("Message in polynomial:", message_poly)

  t_T = numpy.transpose(t)
  A_T = numpy.transpose(A)

  #calc u, v
  u = numpy.add(numpy.matmul(A_T, r), e_1)
  #v = numpy.add(numpy.matmul(t_T, r), e_2, message_poly)
  temp = numpy.matmul(t_T, r).flatten().tolist()[0]
  v = temp[0] + e_2 + message_poly

  temp = u.transpose().flatten().tolist()[0]

  for i in range(k):
    temp[i] = rem(temp[i], f)
    temp[i] = Poly([coeff % q for coeff in temp[i].as_poly().all_coeffs()], x)

  u = numpy.asmatrix(numpy.asanyarray(temp)).transpose()

  v = rem(v, f)
  v = Poly([coeff % q for coeff in v.as_poly().all_coeffs()], x)

  print("Ciphertext u:", u)
  print("Ciphertext v:", v)

  #return
  return(u, v)

In [ ]:
def Kyber_Decaps(q, f, k, x, s, u, v):
  s_T = numpy.transpose(s)
  recovered_message_poly_with_error = numpy.subtract(v, numpy.matmul(s_T, u))
  recovered_message_poly_with_error = recovered_message_poly_with_error.flatten().tolist()[0]
  recovered_message_poly_with_error = recovered_message_poly_with_error[0]

  recovered_message_poly_with_error = rem(recovered_message_poly_with_error, f)
  recovered_message_poly_with_error = Poly([coeff % q for coeff in recovered_message_poly_with_error.as_poly().all_coeffs()], x)

  print("Recovered message in polynomial with errors:", recovered_message_poly_with_error)

  recovered_message_binary_list_with_error = recovered_message_poly_with_error.all_coeffs()
  print("Recovered message in binary list with errors:", recovered_message_binary_list_with_error)

  def round(coeff, q):
    coeff = coeff % q
    if coeff >= q/4 and coeff <= 3*q/4:
      #print("round q/2")
      return math.ceil(q/2)
    else:
      #print("round 0")
      return 0

  recovered_message_binary_list_rounded = []
  recovered_message_binary_list = []
  #print("len", len(recovered_message_binary_list_with_error))

  for i in range(len(recovered_message_binary_list_with_error)):
    #print("test", recovered_message_binary_list_with_error[i])
    #print("list", round(recovered_message_binary_list_with_error[i], q))
    recovered_message_binary_list_rounded.append(round(recovered_message_binary_list_with_error[i], q))
    #print(i, recovered_message_binary_list_rounded[i])
    recovered_message_binary_list.append(str(recovered_message_binary_list_rounded[i] // math.ceil(q/2)))

  recovered_message_binary = ''.join(recovered_message_binary_list)
  print("Recovered message in binary:", recovered_message_binary)

  recovered_message_decimal = int(recovered_message_binary, 2)

  #return
  return(recovered_message_decimal)

In [ ]:
#input modulus q
q = int(input("Enter the modulus q: "))

#input polynomial modulus f
f = Poly(list(map(int, input("Enter coefficients of f(x) in increasing order of degree separated by space: ").split())), symbols('x'))
print("Plynomial modulus f:", f)

#All the vectors are of the same size including the dimension of square matrix A
k = int(input("Enter the size of vecors/matrix used overall: "))

x = symbols('x')  # Define the symbol for the polynomial

#Call func KeyGen which returns vector t
s, A, t = Kyber_KeyGen(q, f, k, x)


#print public key (A, t)
print("Public Key (A, t):", (A, t))

#print private key (s)
print("Private Key (s):", s)

Enter the modulus q: 17
Enter coefficients of f(x) in increasing order of degree separated by space: 1 0 0 0 1
Plynomial modulus f: Poly(x**4 + 1, x, domain='ZZ')
Enter the size of vecors/matrix used overall: 2 
Enter coefficients in decreasing order of degree separated by space - s_0: -1 -1 1 0
Enter coefficients in decreasing order of degree separated by space - s_1: -1 0 -1 0
Private Key as list: [Poly(-x**3 - x**2 + x, x, domain='ZZ'), Poly(-x**3 - x, x, domain='ZZ')]
Private Key: [[Poly(-x**3 - x**2 + x, x, domain='ZZ')]
 [Poly(-x**3 - x, x, domain='ZZ')]]
Enter coefficients in decreasing order of degree separated by space - Ar_00: 6 16 16 11
Enter coefficients in decreasing order of degree separated by space - Ar_01: 9 4 6 3 
Enter coefficients in decreasing order of degree separated by space - Ar_10: 5 3 10 1
Enter coefficients in decreasing order of degree separated by space - Ar_11: 6 1 9 15
Matrix A as list: [[Poly(6*x**3 + 16*x**2 + 16*x + 11, x, domain='ZZ'), Poly(9*x**3 + 

In [ ]:
#Call func Enc which returns ciphertext (u, v)
u, v = Kyber_Encaps(q, f, k, x, A, t)

#print ciphertext (u, v)
print("Ciphertext (u, v):", (u, v))

Enter coefficients in decreasing order of degree separated by space - r_0: -1 1 0 0
Enter coefficients in decreasing order of degree separated by space - r_1: 1 1 0 -1
Randomizer polynomial vector r as list: [Poly(-x**3 + x**2, x, domain='ZZ'), Poly(x**3 + x**2 - 1, x, domain='ZZ')]
Randomizer polynomial vector r: [[Poly(-x**3 + x**2, x, domain='ZZ')]
 [Poly(x**3 + x**2 - 1, x, domain='ZZ')]]
Enter coefficients in decreasing order of degree separated by space - e_1_0: 1 1 0
Enter coefficients in decreasing order of degree separated by space - e_1_1: 1 0 0
Error vector e_1 as list: [Poly(x**2 + x, x, domain='ZZ'), Poly(x**2, x, domain='ZZ')]
Error vector e_1: [[Poly(x**2 + x, x, domain='ZZ')]
 [Poly(x**2, x, domain='ZZ')]]
Enter coefficients in decreasing order of degree separated by space - e_2_1: -1 -1 0 0
Enter message in decimal to encrypt: 5
Message in binary: [1, 0, 1]
Message in binary multiplied by ceil(q/2): [9, 0, 9]
Message in polynomial: Poly(9*x**2 + 9, x, domain='ZZ')
Ciph

In [ ]:
#Call func Dec with parameters (s, ciphertext) which returns message in decimal format
decrypted_message = Kyber_Decaps(q, f, k, x, s, u, v)

#print decrypted message (decimal)
print("Decrypted message (decimal):", decrypted_message)

Recovered message in polynomial with errors: Poly(16*x**3 + 6*x**2 + 16*x + 6, x, domain='ZZ')
Recovered message in binary list with errors: [16, 6, 16, 6]
Recovered message in binary: 0101
Decrypted message (decimal): 5


Inputs      

q = 17     
f = 1 0 0 0 1      
k = 2     
s = -1 -1 1 0 || -1 0 -1 0       
A = 6 16 16 11 || 9 4 6 3 || 5 3 10 1 || 6 1 9 15         
e = 1 0 0 || 1 -1 0          
r = -1 1 0 0 || 1 1 0 -1            
e_1 = 1 1 0 || 1 0 0          
e_2 = -1 -1 0 0          
m = 11


